<a href="https://colab.research.google.com/github/explrA/Precog-Tasks/blob/main/Task2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Task 2.b:**
Sentence Similarity with Dependency Parsing using Average of Token Embeddings

In [1]:
!pip install gensim
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [2]:
from datasets import load_dataset
from gensim.downloader import load
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import random
import pandas as pd
import numpy as np
import spacy

ds = load_dataset("google-research-datasets/paws", "labeled_final")
word2vec_model = load('word2vec-google-news-300')

dp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [3]:
random.seed(45)

train_1 = ds['train'].shuffle(seed=45).select(range(int(1.0 * len(ds['train']))))
dev_1 = ds['validation'].shuffle(seed=45).select(range(int(1.0 * len(ds['validation']))))
test_1 = ds['test'].shuffle(seed=45).select(range(int(1.0 *len(ds['test']))))

train_df = pd.DataFrame(train_1)
dev_df = pd.DataFrame(dev_1)
test_df = pd.DataFrame(test_1)

In [4]:
def dp_embed(sent, w2v_model, dp):

  dp_sent = dp(sent)
  dp_embeds = []

  for word in dp_sent:
    if word.dep_ in {"nsubj", "dobj", "ROOT"} and word.text in w2v_model:
      dp_embeds.append(w2v_model[word.text])

  if not dp_embeds:
    return np.zeros(w2v_model.vector_size)

  return np.mean(dp_embeds, axis=0)


def cosine_sim(sent1, sent2, w2v_model, dp):

  sent_embed1 = dp_embed(sent1, w2v_model, dp)
  sent_embed2 = dp_embed(sent2, w2v_model, dp)

  sent_similarity = cosine_similarity([sent_embed1], [sent_embed2]) [0][0]

  return sent_similarity

In [5]:
train_df['cal_sim'] = train_df.apply(lambda i: cosine_sim(i['sentence1'], i['sentence2'], word2vec_model, dp), axis=1)
train_sim = train_df['cal_sim'].values
train_labels = train_df['label'].values
train_sim_2d = np.array(train_sim).reshape(-1, 1)

dev_df['cal_sim'] = dev_df.apply(lambda j: cosine_sim(j['sentence1'], j['sentence2'], word2vec_model, dp), axis=1)
dev_sim = dev_df['cal_sim'].values
dev_labels = dev_df['label'].values
dev_sim_2d = np.array(dev_sim).reshape(-1, 1)


classifier = LogisticRegression()
classifier.fit(train_sim_2d, train_labels)

dev_pred = classifier.predict(dev_sim_2d)
dev_acc = accuracy_score(dev_labels, dev_pred)
print("dev acc: ", dev_acc)

dev acc:  0.557625


In [6]:
test_df['cal_sim'] = test_df.apply(lambda j: cosine_sim(j['sentence1'], j['sentence2'], word2vec_model, dp), axis=1)
test_sim = test_df['cal_sim'].values
test_labels = test_df['label'].values
test_sim_2d = np.array(test_sim).reshape(-1, 1)

test_pred = classifier.predict(test_sim_2d)
test_acc = accuracy_score(test_labels, test_pred)
print("test_acc: ",test_acc)

test_acc:  0.558
